This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/robot.html).  I recommend having both windows open, side-by-side!

In [7]:
import numpy as np
from pydrake.all import PackageMap, Parser, RobotDiagramBuilder, Simulator, StartMeshcat

from manipulation import running_as_notebook
from manipulation.make_drake_compatible_model import MakeDrakeCompatibleModel
from manipulation.remotes import AddGymnasiumRobotics
from manipulation.utils import ApplyDefaultVisualization

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

## Gymnasium Robotics

Includes the following environments:
- Franka kitchen
- ...

In [ ]:
# TODO(russt): Use ModelVisualizer pending resolution of Drake#22444.
if running_as_notebook:  # I don't want to download the repo in CI.
    meshcat.Delete()
    builder = RobotDiagramBuilder()
    plant = builder.plant()
    parser = Parser(plant)
    package_map = parser.package_map()
    AddGymnasiumRobotics(package_map)
    original_file = package_map.ResolveUrl(
        "package://gymnasium_robotics/kitchen_franka/kitchen_assets/kitchen_env_model.xml"
    )
    drake_compatible_file = original_file.replace(".xml", ".drake.xml")
    MakeDrakeCompatibleModel(original_file, drake_compatible_file)
    model_instances = parser.AddModels(drake_compatible_file)
    plant.Finalize()
    ApplyDefaultVisualization(builder.builder(), meshcat=meshcat)
    diagram = builder.Build()
    simulator = Simulator(diagram)

    # Workaround for drake#22444: set the desired state to zero.
    for model_instance in model_instances:
        desired_state_port = plant.get_desired_state_input_port(model_instance)
        if desired_state_port.size() > 0:
            desired_state_port.FixValue(
                plant.GetMyContextFromRoot(simulator.get_context()),
                np.zeros(desired_state_port.size()),
            )

    diagram.ForcedPublish(simulator.get_context())